# Signal vs. background classification with NEW full MC

In [ ]:
%matplotlib inline
from matplotlib.patches         import Ellipse
import matplotlib.pyplot as plt
import numpy  as np
import random as rd
import tables as tb

from __future__  import print_function
from scipy.stats import threshold

from keras.models               import Model
from keras.layers               import Input, Dense, Convolution3D, Activation, Dropout
from keras.optimizers           import SGD, Adam, Nadam         
#from keras.callbacks            import ReduceLROnPlateau !!!!!
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core          import Flatten
from keras                      import callbacks

Load Data

In [ ]:
Ntrain = 8000     # number of training events per sample

# Signal events.
s_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_si.h5', 'r')
print(s_dat)
s_array = np.array(s_dat.root.maps)
x_t = s_array[:Ntrain]
x_v = s_array[Ntrain:]
y_t = np.zeros([Ntrain, 2])
y_t[:,0] += 1
y_v = np.zeros([len(s_array)-Ntrain, 2])
y_v[:,0] += 1

# Add background events.
b_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_bg.h5', 'r')
print(b_dat)
b_array = np.array(b_dat.root.maps)
x_t = np.concatenate([x_t, b_array[:Ntrain]])
x_v = np.concatenate([x_v, b_array[Ntrain:]])
y_bt = np.zeros([Ntrain, 2])
y_bt[:,1] += 1
y_t = np.concatenate([y_t, y_bt])
y_bv = np.zeros([len(b_array)-Ntrain, 2])
y_bv[:,1] += 1
y_v = np.concatenate([y_v, y_bv])

# Include the final dimension (single-channel).
x_t = np.reshape(x_t, (len(x_t), 48, 48, 30, 1))
x_v = np.reshape(x_v, (len(x_v), 48, 48, 30, 1))
print("Prepared", len(x_t), "training events and", len(x_v), "validation events.")

In [ ]:
# Set up a DNN for 3D convolution
nsipm = int(48)
nslices = int(30)

inputs = Input(shape=(48, 48, 30, 1))
cinputs = Convolution3D(32, 6, 6, 6, border_mode='same', subsample=(1, 1, 1), activation='relu')(inputs)
cinputs = Convolution3D(64, 6, 6, 6, border_mode='valid', subsample=(2, 2, 3), activation='relu')(cinputs)
cinputs = Convolution3D(128, 3, 3, 1, border_mode='valid', subsample=(2, 2, 1), activation='relu')(cinputs)
cinputs = Convolution3D(256, 1, 1, 1, border_mode='valid', subsample=(2, 2, 2), activation='relu')(cinputs)
cinputs = Convolution3D(512, 2, 2, 1, border_mode='valid', subsample=(2, 2, 5), activation='relu')(cinputs)
cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)
f1 = Flatten()(cinputs)
f1 = Dense(output_dim=512, activation='relu')(f1)
# f1 = Dropout(.3)(f1)

inc_output = Dense(output_dim=2, activation='sigmoid')(f1)
incep = Model(inputs, inc_output)

incep.compile(loss='categorical_crossentropy',
              optimizer=Nadam(lr=0.01, beta_1=0.9, beta_2=0.999,
                              epsilon=1e-08, schedule_decay=0.004),
              metrics=['accuracy'])

incep.summary()

# Train the network
Ntrain = 8000
lcallbacks = [callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min'), 
              callbacks.ModelCheckpoint('models/conv3d_200k.h', monitor='val_loss', save_best_only=True, mode='min')]

In [ ]:
hist = incep.fit(x_t, y_t, shuffle=True, nb_epoch=40, batch_size=500, verbose=1, validation_data=(x_v, y_v))

In [ ]:
# Carried over from NEW_kr_diff_mc_train.ipynb
def NEW_SiPM_map_plot(xarr, yarr, plot_truth=True, normalize=True):
    """
    Plots a SiPM map in the NEW Geometry
    xarr is a NEW sipm map, yarr the pair of coordinates the map corresponds to
    """
    if normalize:
        probs = (xarr - np.min(xarr))
        probs /= np.max(probs)
    else: 
        probs = xarr

    for i in range(48):
        for j in range(48):
            r = Ellipse(xy=(i * 10 - 235, j * 10 - 235), width=2., height=2.);
            r.set_facecolor('0');
            r.set_alpha(probs[i, j]);
            ax1.add_artist(r);
            
    if plot_truth:
        # Place a large blue circle for actual EL points.
        xpt = yarr[0]
        ypt = yarr[1]
        mrk = Ellipse(xy=(xpt,ypt), width=4., height=4.);
        mrk.set_facecolor('b');
        ax1.add_artist(mrk);
        #print(xpt,ypt)

NEW_SiPM_map_plot(xarr, yarr)        
    
# Show the plot.
plt.xlabel("x (mm)");
plt.ylabel("y (mm)");
#plt.show()